# Raster-based market access

Following in the footsteps of the Malaria Atlas Project's <a href="https://developers.google.com/earth-engine/datasets/catalog/Oxford_MAP_friction_surface_2015_v1_0">Global Friction Surface</a>, this notebook explores a set of tools for calculating:

1. Travel time rasters
2. Drive-time polygons
3. Marketsheds


In [2]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../src")

import GOSTNetsRaster.market_access as ma

In [5]:
tutorial_folder = os.path.join("../", "tutorial_data")
dests = os.path.join(tutorial_folder, "destinations.shp")
friction_surface = os.path.join(tutorial_folder, "global_friction_surface.tif")


inD = gpd.read_file(dests)
inR = rasterio.open(friction_surface)
frictionD = inR.read()[0,:,:]
# convert friction surface to traversal time (lazily). Original data are
#    the original data are minutes to travel 1 m, so we will convert to 
#    minutes to cross the cell
frictionD = frictionD * 1000
mcp = graph.MCP_Geometric(frictionD)

DriverError: ../tutorial_data/destinations.shp: No such file or directory

In [3]:
ma.calculate_travel_time?

In [ ]:
travel_costs, traceback = ma.calculate_travel_time(inR, mcp, inD)
travel_costs = travel_costs.astype(inR.meta['dtype'])
with rasterio.open(os.path.join(tutorial_folder, "least_cost_travel_time.tif"), 'w', **inR.meta) as out_file:
    out_file.write_band(1, travel_costs)

In [ ]:
ma.generate_feature_vectors?

In [ ]:
drive_time_thresholds = [60, 120, 180, 240] # minutes
drive_vectors = ma.generate_feature_vectors(inR, mcp, inD, drive_time_thresholds)
drive_vectors.to_file(os.path.join(tutorial_folder, "drive_vectors.shp"))

In [ ]:
ma.generate_market_sheds?

In [ ]:
importlib.reload(ma)
outfile = os.path.join(tutorial_folder, "marketsheds.tif")
ma.generate_market_sheds(inR, inD, out_file=outfile)